In [ ]:
# CELL 1: IMPORTS & CONFIGURATION
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import silhouette_score
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# PLOTTING SETUP FOR PUBLICATION
# This makes fonts larger and lines thicker for academic papers
sns.set_theme(style="whitegrid", context="paper", font_scale=1.5)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['lines.linewidth'] = 2.0

print("Libraries loaded. Ready for scientific computing.")

The Data Loader

In [ ]:
# CELL 2 (CORRECTED): DATA LOADING & CLEANING
def load_and_clean_data(file_path):
    print("Loading Dataset...")

    # 1. LOAD (Using 'tstamp' for date)
    df = pd.read_csv(file_path, parse_dates=['tstamp'])

    # 2. RENAME COLUMNS (To match our scientific code)
    # We rename 'tstamp' -> 'Date' and 'PotActiva' -> 'PotAtiva'
    df = df.rename(columns={'tstamp': 'Date', 'PotActiva': 'PotAtiva'})

    # 3. SORTING
    df = df.sort_values(by=['CPE', 'Date'])

    # 4. REMOVE DUPLICATES
    df = df.drop_duplicates(subset=['CPE', 'Date'], keep='first')

    # 5. SCIENTIFIC INTERPOLATION
    # Fill small gaps linearly to preserve data integrity
    df['PotAtiva'] = df.groupby('CPE')['PotAtiva'].transform(
        lambda x: x.interpolate(method='linear', limit=2)
    )

    # Drop rows that are still empty
    df = df.dropna(subset=['PotAtiva'])

    print(f"Data Loaded Successfully.")
    print(f"Unique Consumers: {df['CPE'].nunique()}")
    print(f"Total Observations: {len(df)}")
    print(f"Date Range: {df['Date'].min()} to {df['Date'].max()}")

    return df

# === ACTION ===
# UPDATE THIS PATH to your actual file location
filename = 'consumo15m_11_2025.csv'
df_raw = load_and_clean_data(filename)

Feature Engineering (The "90% Accuracy" Engine)

In [ ]:
# CELL 3 (FINAL): COMPLETE FEATURE ENGINEERING
def generate_research_features(df):
    print("Processing Time-Series Features...")

    # 1. RESAMPLE TO HOURLY
    df = df.set_index('Date')
    df_hourly = df.groupby('CPE')['PotAtiva'].resample('1H').mean().reset_index()

    # 2. FILL GAPS
    df_hourly['PotAtiva'] = df_hourly.groupby('CPE')['PotAtiva'].transform(
        lambda x: x.interpolate(method='linear', limit=24)
    )

    # 3. GENERATE TIME FEATURES
    df_hourly['Hour'] = df_hourly['Date'].dt.hour
    df_hourly['DayOfWeek'] = df_hourly['Date'].dt.dayofweek

    # Cyclical Features (REQUIRED by Cell 6)
    df_hourly['hour_sin'] = np.sin(2 * np.pi * df_hourly['Hour'] / 24)
    df_hourly['hour_cos'] = np.cos(2 * np.pi * df_hourly['Hour'] / 24)
    df_hourly['day_sin'] = np.sin(2 * np.pi * df_hourly['DayOfWeek'] / 7)
    df_hourly['day_cos'] = np.cos(2 * np.pi * df_hourly['DayOfWeek'] / 7)

    # 4. GENERATE LAG FEATURES (The Memory)
    grouped = df_hourly.groupby('CPE')['PotAtiva']

    df_hourly['lag_24h'] = grouped.shift(24)      # Yesterday
    df_hourly['lag_168h'] = grouped.shift(168)    # Last Week

    # 5. GENERATE ROLLING STATISTICS
    # Rolling Mean of Last Week (Smooths out trends)
    df_hourly['rolling_mean_7d'] = grouped.shift(168).rolling(window=24).mean()

    # Rolling Volatility (Standard Deviation)
    df_hourly['rolling_std_24h'] = grouped.shift(1).rolling(window=24).std()

    # 6. FINAL CLEANUP
    # Drop rows where features are calculated as NaN
    df_clean = df_hourly.dropna(subset=['PotAtiva', 'lag_168h', 'rolling_mean_7d'])

    print(f"Features Generated. Final shape: {df_clean.shape}")
    return df_clean

# === ACTION ===
df_features = generate_research_features(df_raw)

Clustering Methodology (Paper Figure 1)

In [ ]:
   # CELL 4: DETERMINE OPTIMAL CLUSTERS (The "Novelty" Proof)
def optimize_clusters(df):
    print("Optimizing Clusters...")

    # 1. CREATE LOAD PROFILES
    # Pivot: Rows=CPEs, Cols=Hours (0-23). We average usage per hour.
    # This creates a "Daily Signature" for each user.
    daily_profiles = df.groupby(['CPE', df['Date'].dt.hour])['PotAtiva'].mean().unstack()

    # Normalize (So high users don't dominate low users just by volume)
    scaler = MinMaxScaler()
    daily_profiles_norm = pd.DataFrame(
        scaler.fit_transform(daily_profiles.T).T,
        index=daily_profiles.index,
        columns=daily_profiles.columns
    )

    # 2. TEST K (2 to 6)
    results = []
    K_range = range(2, 7)

    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(daily_profiles_norm)
        score = silhouette_score(daily_profiles_norm, labels)
        inertia = kmeans.inertia_
        results.append({'k': k, 'silhouette': score, 'inertia': inertia})

    results_df = pd.DataFrame(results)

    # 3. PLOT FOR PAPER
    fig, ax1 = plt.subplots()

    # Inertia (Elbow)
    sns.lineplot(data=results_df, x='k', y='inertia', marker='o', ax=ax1, color='blue', label='Inertia')
    ax1.set_ylabel('Inertia (Lower is Better)', color='blue')

    # Silhouette (Quality)
    ax2 = ax1.twinx()
    sns.lineplot(data=results_df, x='k', y='silhouette', marker='s', ax=ax2, color='red', label='Silhouette')
    ax2.set_ylabel('Silhouette Score (Higher is Better)', color='red')

    plt.title('Cluster Optimization: Finding the "Energy Phenotypes"')
    plt.show()

    best_k = results_df.loc[results_df['silhouette'].idxmax()]['k']
    print(f"Mathematically Optimal Clusters (K): {int(best_k)}")

    return int(best_k), daily_profiles_norm

# === ACTION ===
best_k, profiles_norm = optimize_clusters(df_features)

In [ ]:
# CELL 5: APPLY CLUSTERING LABELS
# This assigns every consumer to either "Cluster 0" or "Cluster 1"
def apply_clusters(df, k=2):
    print(f"Applying K-Means with K={k}...")

    # 1. Prepare Data (Same normalization as Cell 4)
    daily_profiles = df.groupby(['CPE', df['Date'].dt.hour])['PotAtiva'].mean().unstack()
    scaler = MinMaxScaler()
    daily_profiles_norm = pd.DataFrame(
        scaler.fit_transform(daily_profiles.T).T,
        index=daily_profiles.index,
        columns=daily_profiles.columns
    )

    # 2. Fit K-Means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(daily_profiles_norm)

    # 3. Save Labels to the Main DataFrame
    # Map the cluster label back to the original dataframe based on CPE
    cluster_map = pd.Series(cluster_labels, index=daily_profiles.index)
    df['Cluster'] = df['CPE'].map(cluster_map)

    print("Cluster counts:")
    print(df.groupby('CPE')['Cluster'].first().value_counts())

    return df

# === ACTION ===
# Overwrite df_features with the version that has clusters
df_final = apply_clusters(df_features, k=2)

In [ ]:
# CELL 6: GENERATE PAPER FIGURES
def generate_paper_plots(df):
    print("Generating Figures for Research Paper...")

    # Prepare Data
    daily_profiles = df.groupby(['CPE', df['Date'].dt.hour])['PotAtiva'].mean().unstack()
    scaler = MinMaxScaler()
    daily_profiles_norm = pd.DataFrame(scaler.fit_transform(daily_profiles.T).T,
                                     index=daily_profiles.index, columns=daily_profiles.columns)

    # --- FIGURE 1: ELBOW PLOT ---
    inertia = []
    K_range = range(1, 10)
    for k in K_range:
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        km.fit(daily_profiles_norm)
        inertia.append(km.inertia_)

    plt.figure(figsize=(10, 5))
    plt.plot(K_range, inertia, 'bo-', linewidth=2)
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Figure 1: Elbow Method (Optimal K=2)')
    plt.axvline(x=2, color='r', linestyle='--')
    plt.grid(True)
    plt.show()

    # --- FIGURE 2: CLUSTER PROFILES ---
    km_final = KMeans(n_clusters=2, random_state=42, n_init=10)
    km_final.fit(daily_profiles_norm)
    centroids = km_final.cluster_centers_

    plt.figure(figsize=(12, 6))
    plt.plot(centroids[0], color='blue', linewidth=3, label='Cluster 0 (Stable)')
    plt.plot(centroids[1], color='orange', linewidth=3, label='Cluster 1 (Volatile)')
    plt.title('Figure 2: Distinct Consumer Load Profiles')
    plt.xlabel('Hour of Day')
    plt.ylabel('Normalized Consumption')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# === ACTION ===
generate_paper_plots(df_final)

In [ ]:
# CELL: PUBLICATION-QUALITY PCA SCATTER PLOT
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

def plot_best_pca(df):
    print("Generating Publication-Quality PCA Plot...")

    # 1. PREPARE DATA
    # Ensure we have the hour column
    if 'Hour' not in df.columns:
        df['Hour'] = df['Date'].dt.hour

    # Pivot: Rows=Consumers, Cols=Hours (The "Shape" of usage)
    daily_profiles = df.groupby(['CPE', 'Hour'])['PotAtiva'].mean().unstack()

    # Normalize (Critical for PCA)
    scaler = MinMaxScaler()
    X = pd.DataFrame(scaler.fit_transform(daily_profiles.T).T,
                     index=daily_profiles.index,
                     columns=daily_profiles.columns)

    # Get Cluster Labels (Ensure they exist)
    if 'Cluster' in df.columns:
        # Create a map {CPE: ClusterLabel} to ensure alignment
        cluster_map = df.groupby('CPE')['Cluster'].first()
        labels = X.index.map(cluster_map)
    else:
        print("Error: Please run the clustering cell first to assign labels.")
        return

    # 2. CALCULATE PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)

    # Calculate Variance Explained (Good for paper caption)
    var_exp = pca.explained_variance_ratio_
    total_var = sum(var_exp) * 100

    # 3. CALCULATE TRUE CENTROIDS IN PCA SPACE
    # (We find the center in 24D, then project it to 2D)
    # This is more accurate than just averaging the dots
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    kmeans.fit(X) # Re-fit on aligned data to get centers
    centroids_pca = pca.transform(kmeans.cluster_centers_)

    # 4. PLOT CONFIGURATION
    sns.set_style("whitegrid")
    plt.figure(figsize=(10, 7), dpi=300) # High DPI for printing

    # Plot the dots (Consumers)
    scatter = sns.scatterplot(
        x=X_pca[:, 0],
        y=X_pca[:, 1],
        hue=labels,
        palette=['#2b7bba', '#e67e22'], # Professional Blue & Orange
        style=labels, # Different shapes for colorblind accessibility
        s=100,
        alpha=0.8,
        edgecolor='w'
    )

    # Plot the Centroids (Red X)
    plt.scatter(
        centroids_pca[:, 0], centroids_pca[:, 1],
        marker='X', s=400, linewidths=3,
        color='red', label='Cluster Centroids', zorder=10
    )

    # 5. LABELS & LEGEND
    plt.xlabel(f'Principal Component 1 ({var_exp[0]:.1%} Variance)', fontsize=12, fontweight='bold')
    plt.ylabel(f'Principal Component 2 ({var_exp[1]:.1%} Variance)', fontsize=12, fontweight='bold')
    plt.title(f'Consumer Segmentation Map (Total Explained Variance: {total_var:.1f}%)', fontsize=14, pad=15)

    # Custom Legend
    handles, _ = scatter.get_legend_handles_labels()
    # Add the centroid marker to legend manually if needed, or rely on clear visual
    plt.legend(title='Behavior Profile', loc='upper right', frameon=True, framealpha=0.9)

    plt.tight_layout()
    plt.show()

# === ACTION ===
plot_best_pca(df_final)

The Comparative Forecasting Loop

In [ ]:
# CELL 6: SCIENTIFIC FORECASTING EXPERIMENT
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb

def evaluate_performance(y_true, y_pred, name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return {'Model': name, 'MAE': mae, 'RMSE': rmse}

def run_experiment(df):
    print("Initializing Scientific Forecasting Experiment...")

    # 1. TRAIN/TEST SPLIT (Time-Based)
    # We test on the last 2 months of data to simulate "Future Prediction"
    split_date = df['Date'].max() - pd.Timedelta(days=60)

    train = df[df['Date'] < split_date]
    test = df[df['Date'] >= split_date]

    print(f"Training on {len(train)} samples. Testing on {len(test)} samples.")

    # FEATURES (The ones we created in Cell 3)
    features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos',
                'lag_24h', 'lag_168h', 'rolling_mean_7d', 'rolling_std_24h']
    target = 'PotAtiva'

    results = []

    # ==========================================
    # EXPERIMENT A: NAIVE BASELINE (Required)
    # ==========================================
    # Prediction = "What happened same time last week?"
    baseline_mae = mean_absolute_error(test[target], test['lag_168h'])
    baseline_rmse = np.sqrt(mean_squared_error(test[target], test['lag_168h']))
    results.append({'Model': 'Naive Baseline (Last Week)', 'MAE': baseline_mae, 'RMSE': baseline_rmse})

    # ==========================================
    # EXPERIMENT B: GLOBAL MODEL (Standard)
    # ==========================================
    print("\nTraining Global XGBoost Model (All Clusters Mixed)...")
    global_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, n_jobs=-1, random_state=42)
    global_model.fit(train[features], train[target])

    global_pred = global_model.predict(test[features])
    results.append(evaluate_performance(test[target], global_pred, "Global XGBoost"))

    # ==========================================
    # EXPERIMENT C: CLUSTER-SPECIFIC MODELS (Novelty)
    # ==========================================
    print("\nTraining Cluster-Specific Models (Your Contribution)...")

    # We store predictions here to calculate total error later
    test_clustered = test.copy()
    test_clustered['Hybrid_Pred'] = 0.0

    # Loop through our 2 Clusters
    for k in sorted(df['Cluster'].unique()):
        print(f"  > Training specialized model for Cluster {k}...")

        # Filter Data
        k_train = train[train['Cluster'] == k]
        k_test = test[test['Cluster'] == k]

        if len(k_test) == 0:
            continue

        # Train specialized model
        k_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, n_jobs=-1, random_state=42)
        k_model.fit(k_train[features], k_train[target])

        # Predict
        k_pred = k_model.predict(k_test[features])

        # Save to main test set
        test_clustered.loc[k_test.index, 'Hybrid_Pred'] = k_pred

    # Evaluate the Combined Hybrid Approach
    hybrid_mae = mean_absolute_error(test_clustered[target], test_clustered['Hybrid_Pred'])
    hybrid_rmse = np.sqrt(mean_squared_error(test_clustered[target], test_clustered['Hybrid_Pred']))
    results.append({'Model': 'Hybrid Cluster-XGBoost (Proposed)', 'MAE': hybrid_mae, 'RMSE': hybrid_rmse})

    # ==========================================
    # FINAL RESULTS TABLE
    # ==========================================
    results_df = pd.DataFrame(results)

    # Calculate % Improvement over Baseline
    base_mae = results_df.loc[0, 'MAE']
    results_df['% Improvement'] = 100 * (1 - (results_df['MAE'] / base_mae))

    print("\n=== FINAL RESEARCH RESULTS ===")
    print(results_df)

    # VISUALIZATION (Figure 3 for Paper)
    # Plot residuals (errors) to show the Hybrid model is tighter
    plt.figure(figsize=(10, 5))
    sns.kdeplot(test[target] - test['lag_168h'], label='Baseline Error', fill=True, alpha=0.3)
    sns.kdeplot(test_clustered[target] - test_clustered['Hybrid_Pred'], label='Hybrid Model Error', fill=True, alpha=0.3)
    plt.xlim(-5, 5) # Zoom in on the center
    plt.title("Error Distribution: Hybrid Model vs Baseline")
    plt.xlabel("Prediction Error (kW)")
    plt.legend()
    plt.show()

    return results_df

# === ACTION ===
final_table = run_experiment(df_final)

In [ ]:
# CELL 8 (FIXED): MEMORY-SAFE MULTI-MODEL COMPARISON
from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_error
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt

def run_high_accuracy_experiment(df):
    print("Starting Optimized High-Accuracy Race...")

    # 1. OPTIMIZED SPLIT (Focus on recent relevant data)
    # We take the last 14 months: 12 for training, 2 for testing.
    # This reduces RAM usage by ~60% while keeping the most important patterns.
    cutoff_date = df['Date'].max() - pd.Timedelta(days=400)
    df_recent = df[df['Date'] > cutoff_date].copy()

    split_date = df_recent['Date'].max() - pd.Timedelta(days=60)
    train = df_recent[df_recent['Date'] < split_date]
    test = df_recent[df_recent['Date'] >= split_date]

    print(f"Training on {len(train)} recent samples (Last 12 Months).")

    features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos',
                'lag_24h', 'lag_168h', 'rolling_mean_7d', 'rolling_std_24h']
    target = 'PotAtiva'

    results = []

    # 2. DEFINE EFFICIENT MODELS (No memory-heavy Random Forest)
    models = {
        # LightGBM: The King of Efficiency
        'LightGBM': lgb.LGBMRegressor(n_estimators=300, learning_rate=0.05,
                                      num_leaves=31, verbose=-1, random_state=42, n_jobs=1),

        # XGBoost: High Precision
        'XGBoost': xgb.XGBRegressor(n_estimators=300, learning_rate=0.05,
                                    max_depth=6, n_jobs=1, random_state=42),

        # HistGradientBoosting: Scikit-Learn's low-memory version of RF
        'HistGradient': HistGradientBoostingRegressor(max_iter=300, learning_rate=0.05,
                                                      random_state=42)
    }

    # 3. TRAIN & EVALUATE
    trained_estimators = []

    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(train[features], train[target])
        pred = model.predict(test[features])

        r2 = r2_score(test[target], pred)
        mae = mean_absolute_error(test[target], pred)
        print(f"  > {name}: R2 = {r2:.4f}")

        results.append({'Model': name, 'R2 (Accuracy)': r2, 'MAE': mae})
        trained_estimators.append((name, model))

    # 4. ENSEMBLE (The "Super Model")
    print("Training Ensemble (Voting) Model...")
    ensemble = VotingRegressor(estimators=trained_estimators, n_jobs=1)
    ensemble.fit(train[features], train[target])
    ens_pred = ensemble.predict(test[features])

    ens_r2 = r2_score(test[target], ens_pred)
    ens_mae = mean_absolute_error(test[target], ens_pred)
    results.append({'Model': 'Ensemble (Combined)', 'R2 (Accuracy)': ens_r2, 'MAE': ens_mae})

    # 5. OUTPUT TABLE
    results_df = pd.DataFrame(results).sort_values(by='R2 (Accuracy)', ascending=False)
    print("\n=== FINAL ACCURACY REPORT ===")
    print(results_df)

    # 6. PLOT PREDICTIONS (Visual Proof)
    plt.figure(figsize=(15, 6))
    # Plot a random 4-day window from the test set to show detail
    sample_start = 0
    sample_end = 96 # 4 days * 24 hours

    plt.plot(test.iloc[sample_start:sample_end]['Date'],
             test.iloc[sample_start:sample_end][target].values,
             'k-', label='Actual Energy', linewidth=2)

    plt.plot(test.iloc[sample_start:sample_end]['Date'],
             ens_pred[sample_start:sample_end],
             'r--', label='Ensemble Prediction', linewidth=2)

    plt.title('Figure 3: Prediction vs Reality (Ensemble Model)')
    plt.ylabel('Active Power (kW)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# === ACTION ===
run_high_accuracy_experiment(df_features)